# Converting from Torch to ONNX

ONNX Runtime is a performance-focused engine for ONNX models, which inferences efficiently across multiple platforms and hardware (Windows, Linux, and Mac and on both CPUs and GPUs). ONNX Runtime has proved to considerably increase performance over multiple models as explained here

For this tutorial, you will need to install ONNX and ONNX Runtime. You can get binary builds of ONNX and ONNX Runtime with pip install onnx onnxruntime. Note that ONNX Runtime is compatible with Python versions 3.5 to 3.7.

In [0]:
# Some standard imports
import io
import numpy as np
import torch.nn as nn
import torch.nn.init as init
# from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx

# Super Resolution using Neural Networks

Super-resolution is a way of increasing the resolution of images, videos and is widely used in image processing or video editing. For this tutorial, we will use a small super-resolution model.

First, let’s create a SuperResolution model in PyTorch. This model uses the efficient sub-pixel convolution layer described in “Real-Time Single Image and Video Super-Resolution Using an Efficient Sub-Pixel Convolutional Neural Network” - Shi et al for increasing the resolution of an image by an upscale factor. The model expects the Y component of the YCbCr of an image as an input, and outputs the upscaled Y component in super resolution.

In [0]:
class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor, inplace=False):
        super().__init__()
        self.relu = nn.ReLU(inplace=inplace)
        self.conv1 = nn.Conv2d(1, 64, (5, 5), (1, 1), (2, 2))
        self.conv2 = nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
        self.conv4 = nn.Conv2d(32, upscale_factor ** 2, (3, 3), (1, 1), (1, 1))
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

        self._initialize_weights()
    
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pixel_shuffle(self.conv4(x))
        return x
    
    def _initialize_weights(self):
        init.orthogonal_(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv4.weight)

In [0]:
#  Create the super-resolution model by using the above model definition.
torch_model = SuperResolutionNet(upscale_factor=3)

Ordinarily, you would now train this model; however, for this tutorial, we will instead download some pre-trained weights. Note that this model was not trained fully for good accuracy and is used here for demonstration purposes only.

It is important to call torch_model.eval() or torch_model.train(False) before exporting the model, to turn the model to inference mode. This is required since operators like dropout or batchnorm behave differently in inference and training mode.

In [0]:
# Load pretrained model weights
model_url = 'https://s3.amazonaws.com/pytorch/test_data/export/superres_epoch100-44c6958e.pth'
batch_size = 1    # just a random number    

In [0]:
# Initialize model with the pretrained weights
map_location = lambda storage, loc: storage

In [0]:
if torch.cuda.is_available():
    map_location = None
torch_model.load_state_dict(model_zoo.load_url(model_url, map_location=map_location))

In [0]:
# set the model to inference mode
torch_model.eval()

 # Trace and Export the model

Exporting a model in PyTorch works via tracing or scripting. 

This tutorial will use as an example a model exported by tracing. To export a model, we call the torch.onnx.export() function. 

This will execute the model, recording a trace of what operators are used to compute the outputs. Because export runs the model, we need to provide an input tensor x. 

The values in this can be random as long as it is the right type and size. Note that the input size will be fixed in the exported ONNX graph for all the input’s dimensions, unless specified as a dynamic axes. 

In this example we export the model with an input of batch_size 1, but then specify the first dimension as dynamic in the dynamic_axes parameter in torch.onnx.export(). 

The exported model will thus accept inputs of size [batch_size, 1, 224, 224] where batch_size can be variable.

In [0]:
# Input to the model
x = torch.randn(batch_size, 1, 224, 224, requires_grad=True)
torch_out = torch_model(x)

In [0]:
# Export the model
torch.onnx.export(torch_model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "super_resolution.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

We also computed torch_out, the output after of the model, which we will use to verify that the model we exported computes the same values when run in ONNX Runtime.

Now let’s compute the output using ONNX Runtime’s Python APIs. This part can normally be done in a separate process or on another machine, but we will continue in the same process so that we can verify that ONNX Runtime and PyTorch are computing the same value for the network.

In order to run the model with ONNX Runtime, we need to create an inference session for the model with the chosen configuration parameters (here we use the default config). Once the session is created, we evaluate the model using the run() api. The output of this call is a list containing the outputs of the model computed by ONNX Runtime.

In [0]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [0]:
print(to_numpy(torch_out))